In [ ]:
import numpy as np
from sklearn.utils import class_weight
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
import os
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import EfficientNetB3

In [ ]:
n_classes = 101
img_width, img_height = 224, 224
batch_size = 16

train_data_dir = 'data-101/train'
validation_data_dir = 'data-101/val'
test_data_dir = 'data-101/test'

In [ ]:

# Count samples
def count_samples(directory):
    total = 0
    for class_name in os.listdir(directory):
        class_dir = os.path.join(directory, class_name)
        total += len(os.listdir(class_dir))
    return total

nb_train_samples = count_samples(train_data_dir)
nb_validation_samples = count_samples(validation_data_dir)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,  
    zoom_range=0.2,   
    rotation_range=20, 
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    brightness_range=[0.9, 1.1],  
    horizontal_flip=True)



test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')


### **Fine tune ResNet50 Pretrained model using Food 101 dataset**

In [2]:
# base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)  # Batch normalization
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)  # Batch normalization
predictions = Dense(n_classes, activation='softmax')(x) 

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers[:-50]:  # Unfreeze more layers
    layer.trainable = False

for layer in base_model.layers[-50:]:
    layer.trainable = True


model.compile(optimizer=Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='working/best_model_3class.hdf5', verbose=1, save_best_only=True)
csv_logger = CSVLogger('working/history_3class.log')

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)


history = model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    epochs=20,
    callbacks=[csv_logger, checkpointer, early_stopping, reduce_lr])
model.save('working/model_trained_3class.hdf5')

Found 480 images belonging to 3 classes.
Found 60 images belonging to 3 classes.


c:\Users\mudar\anaconda3\envs\Tenserflow\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/20
30/30 [==============================] - ETA: 0s - loss: 1.6322 - accuracy: 0.3625
Epoch 1: val_loss improved from inf to 1.13693, saving model to working\best_model_3class.hdf5
30/30 [==============================] - 56s 1s/step - loss: 1.6322 - accuracy: 0.3625 - val_loss: 1.1369 - val_accuracy: 0.2708 - lr: 1.0000e-04
Epoch 2/20
30/30 [==============================] - ETA: 0s - loss: 1.5034 - accuracy: 0.3729
Epoch 2: val_loss improved from 1.13693 to 1.11953, saving model to working\best_model_3class.hdf5
30/30 [==============================] - 38s 1s/step - loss: 1.5034 - accuracy: 0.3729 - val_loss: 1.1195 - val_accuracy: 0.3542 - lr: 1.0000e-04
Epoch 3/20
30/30 [==============================] - ETA: 0s - loss: 1.4680 - accuracy: 0.3896
Epoch 3: val_loss did not improve from 1.11953
30/30 [==============================] - 33s 1s/step - loss: 1.4680 - accuracy: 0.3896 - val_loss: 1.1325 - val_accuracy: 0.3958 - lr: 1.0000e-04
Epoch 4/20
30/30 [=====================

Evaluation

In [ ]:
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)  

test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test accuracy: {test_accuracy}, Test loss: {test_loss}")

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Predict on test data
test_generator.reset()
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

conf_matrix = confusion_matrix(true_classes, predicted_classes)
print(conf_matrix)

report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

def load_and_prep_image(img_path, img_size):
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Reshape for model input
    img_array /= 255.0  
    return img_array

img_path = '.jpg'
img = load_and_prep_image(img_path, (img_width, img_height))
prediction = model.predict(img)
predicted_class = class_labels[np.argmax(prediction)]
print(f"Predicted class: {predicted_class}")

In [ ]:
!pip install openai

In [ ]:
import openai


# openai.api_key =

def get_calories(food_item):
    prompt = f"Write the calories in {food_item} per 100g. The answer must be in the following format: <number> calories. Provide no explanations or anything. You must give the answer in the aforementioned format."
    model_engine = "text-davinci-002"
    completions = openai.Completion.create(engine=model_engine, prompt=prompt, max_tokens=1024, n=1,stop=None,temperature=0.5)
    message = completions.choices[0].text
    return message.strip()

food_item = "banana"
calories = get_calories(food_item)
print(f"The number of calories in an {food_item} is {calories}")

* **Model training on all 101 classes takes some time**
* **It was taking more than an hour for one epoch when the full dataset is used for fine tuning**